# Desempenho de algoritmos de classificação na detecção de intrusão em redes de dispositivos IoT

## Importações

In [ ]:
import glob
import os
import time
from dataclasses import dataclass

import matplotlib.pyplot as plt
import numpy as np
import polars as pl
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

## Variáveis globais

In [ ]:
DATASET_DIRECTORY = "./CICIoT2023/"
GENERATE_DATASETS = False
EIGHT_CLASSES = True

## DTOs

In [ ]:
@dataclass
class ModelDataset:
    x_train: np.ndarray
    y_train: np.ndarray
    x_test: pl.DataFrame
    y_test: np.ndarray

## Funções auxiliares

In [ ]:
def sample_rows(
    group_by: pl.dataframe.group_by.GroupBy, percentage: float
) -> pl.DataFrame:
    dfs = []
    for _, df in group_by:
        if len(df) * percentage <= 1:
            dfs.append(df.sample(n=1))
            continue
        dfs.append(df.sample(fraction=percentage))

    return pl.concat(dfs)


def generate_dataframe(file_list: list, percentage: float) -> pl.DataFrame:
    dfs = []
    for file in file_list:
        df = pl.read_csv(file)
        dfs.append(
            sample_rows(
                group_by=df.group_by(["label"], maintain_order=True),
                percentage=percentage,
            )
        )
    return pl.concat(dfs)

# Criando um dataset menor

In [ ]:
file_dir = "./data/"
if not os.path.exists(file_dir):
    os.makedirs(file_dir)

df_sets = sorted(glob.glob(os.path.join(DATASET_DIRECTORY, "*.csv")))

In [ ]:
if GENERATE_DATASETS:
    generate_dataframe(file_list=df_sets, percentage=0.001).write_csv(
        file="./data/dataset_dev.csv"
    )

In [ ]:
if GENERATE_DATASETS:
    generate_dataframe(file_list=df_sets, percentage=0.01).write_csv(
        file="./data/dataset_one_percent.csv"
    )

# IDS

## Dataset

In [ ]:
df = pl.read_csv("./data/dataset_dev.csv")
training_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

### Separando Features e Label

In [ ]:
X_columns = df.columns[:-1]
y_column = "label"

In [ ]:
del df

### 8 Classes

In [ ]:
if EIGHT_CLASSES:
    eight_classes = {}

    eight_classes["BenignTraffic"] = "Benign"

    eight_classes["DDoS-RSTFINFlood"] = "DDoS"
    eight_classes["DDoS-PSHACK_Flood"] = "DDoS"
    eight_classes["DDoS-SYN_Flood"] = "DDoS"
    eight_classes["DDoS-UDP_Flood"] = "DDoS"
    eight_classes["DDoS-TCP_Flood"] = "DDoS"
    eight_classes["DDoS-ICMP_Flood"] = "DDoS"
    eight_classes["DDoS-SynonymousIP_Flood"] = "DDoS"
    eight_classes["DDoS-ACK_Fragmentation"] = "DDoS"
    eight_classes["DDoS-UDP_Fragmentation"] = "DDoS"
    eight_classes["DDoS-ICMP_Fragmentation"] = "DDoS"
    eight_classes["DDoS-SlowLoris"] = "DDoS"
    eight_classes["DDoS-HTTP_Flood"] = "DDoS"

    eight_classes["DoS-UDP_Flood"] = "DoS"
    eight_classes["DoS-SYN_Flood"] = "DoS"
    eight_classes["DoS-TCP_Flood"] = "DoS"
    eight_classes["DoS-HTTP_Flood"] = "DoS"

    eight_classes["Mirai-greeth_flood"] = "Mirai"
    eight_classes["Mirai-greip_flood"] = "Mirai"
    eight_classes["Mirai-udpplain"] = "Mirai"

    eight_classes["Recon-PingSweep"] = "Recon"
    eight_classes["Recon-OSScan"] = "Recon"
    eight_classes["Recon-PortScan"] = "Recon"
    eight_classes["VulnerabilityScan"] = "Recon"
    eight_classes["Recon-HostDiscovery"] = "Recon"

    eight_classes["DNS_Spoofing"] = "Spoofing"
    eight_classes["MITM-ArpSpoofing"] = "Spoofing"

    eight_classes["BrowserHijacking"] = "Web"
    eight_classes["Backdoor_Malware"] = "Web"
    eight_classes["XSS"] = "Web"
    eight_classes["Uploading_Attack"] = "Web"
    eight_classes["SqlInjection"] = "Web"
    eight_classes["CommandInjection"] = "Web"

    eight_classes["DictionaryBruteForce"] = "BruteForce"

    training_data = training_data.with_columns(
        pl.col(y_column).replace(eight_classes, default=-1)
    )
    test_data = test_data.with_columns(
        pl.col(y_column).replace(eight_classes, default=-1)
    )

## Classes

### Separando as colunas

In [ ]:
X_train = training_data.select(X_columns)
y_train = training_data.select(y_column)
X_test = test_data.select(X_columns)
y_test = test_data.select(y_column).to_numpy()

### Balanceando o dataset

In [ ]:
training_data.group_by("label").len()

In [ ]:
ros = RandomOverSampler(random_state=42)
rus = RandomUnderSampler(random_state=42)

pd_X_resampled, pd_y_resampled = ros.fit_resample(
    X_train.to_pandas(), y_train.to_pandas()
)
X_resampled, y_resampled = pl.from_pandas(pd_X_resampled), pl.from_pandas(
    pd_y_resampled
)

### Normalizando o dataset

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)

# Testando os algoritmos

## Escopo original
Aqui serão executados os algoritmos de classificação Logistic Regression, Perceptron, Adaboost e Random Forest, presentes no trabalho de Neto, E. C. P. *et al.*

In [ ]:
data = ModelDataset(
    x_train=X_train_scaled,
    y_train=y_resampled.to_numpy(),
    x_test=X_test_scaled,
    y_test=y_test,
)

### Logistic Regression

In [ ]:
logistic_regression = LogisticRegression(n_jobs=-1, max_iter=10000)
logistic_regression.fit(data.x_train, data.y_train)
y_pred = logistic_regression.predict(data.x_test)

print(classification_report(data.y_test, y_pred, target_names=np.unique(y_pred)))

accuracy = accuracy_score(data.y_test, y_pred)
precision = precision_score(data.y_test, y_pred, average="macro")
recall = recall_score(data.y_test, y_pred, average="macro")
f1 = f1_score(data.y_test, y_pred, average="macro")

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")

### Perceptron

In [ ]:
# TODO

### AdaBoost

In [ ]:
# TODO

### Random Forest

In [ ]:
random_forest = RandomForestClassifier()
random_forest.fit(data.x_train, data.y_train)
y_pred = random_forest.predict(data.x_test)

print(classification_report(data.y_test, y_pred, target_names=np.unique(y_pred)))

accuracy = accuracy_score(data.y_test, y_pred)
precision = precision_score(data.y_test, y_pred, average="macro")
recall = recall_score(data.y_test, y_pred, average="macro")
f1 = f1_score(data.y_test, y_pred, average="macro")

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")

## Escopo expandido
Aqui serão executados os algoritmos de classificação K-Nearest Neighbors, Support Vector Machine e Gaussian Naive Bayes.

In [ ]:
k_nearest_neighbors = KNeighborsClassifier()
k_nearest_neighbors.fit(data.x_train, data.y_train)
y_pred = k_nearest_neighbors.predict(data.x_test)

print(classification_report(data.y_test, y_pred, target_names=np.unique(y_pred)))

accuracy = accuracy_score(data.y_test, y_pred)
precision = precision_score(data.y_test, y_pred, average="macro")
recall = recall_score(data.y_test, y_pred, average="macro")
f1 = f1_score(data.y_test, y_pred, average="macro")

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")

In [ ]:
support_vector_classification = SVC()
support_vector_classification.fit(data.x_train, data.y_train)
y_pred = support_vector_classification.predict(data.x_test)

print(classification_report(data.y_test, y_pred, target_names=np.unique(y_pred)))

accuracy = accuracy_score(data.y_test, y_pred)
precision = precision_score(data.y_test, y_pred, average="macro")
recall = recall_score(data.y_test, y_pred, average="macro")
f1 = f1_score(data.y_test, y_pred, average="macro")

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")

In [ ]:
gaussian_naive_bayes = GaussianNB()
gaussian_naive_bayes.fit(data.x_train, data.y_train)
y_pred = gaussian_naive_bayes.predict(data.x_test)

print(classification_report(data.y_test, y_pred, target_names=np.unique(y_pred)))

accuracy = accuracy_score(data.y_test, y_pred)
precision = precision_score(data.y_test, y_pred, average="macro")
recall = recall_score(data.y_test, y_pred, average="macro")
f1 = f1_score(data.y_test, y_pred, average="macro")

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")